In [5]:
import pandas as pd

In [6]:
# Read the CSV file
df = pd.read_csv('Romania_Superliga_Players_24_25_adv_stats.csv')

print(f"Original dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Filter rows where generic.position.code (8th column, index 7) equals "CB"
df_filtered = df[df.iloc[:, 7] == 'CB'].copy()
print(f"After filtering for CB position: {df_filtered.shape}")

# Remove specified columns by name
# Note: We'll handle the case where some columns might not exist
columns_to_remove = ['competitionId', 'seasonId', 'positions.percent', 'positions.position.code']
existing_columns_to_remove = [col for col in columns_to_remove if col in df_filtered.columns]

if existing_columns_to_remove:
    df_filtered = df_filtered.drop(columns=existing_columns_to_remove)
    print(f"Removed columns: {existing_columns_to_remove}")
else:
    print("Warning: None of the specified columns to remove were found in the dataset")

print(f"After removing specified columns: {df_filtered.shape}")

# Remove duplicate rows based on playerId, keeping the first occurrence
# First, let's identify the playerId column (assuming it exists)
if 'playerId' in df_filtered.columns:
    initial_count = len(df_filtered)
    df_filtered = df_filtered.drop_duplicates(subset=['playerId'], keep='first')
    duplicates_removed = initial_count - len(df_filtered)
    print(f"Removed {duplicates_removed} duplicate playerId rows")
else:
    print("Warning: 'playerId' column not found. Checking for similar column names...")
    # Look for columns that might contain player IDs
    possible_id_columns = [col for col in df_filtered.columns if 'id' in col.lower() or 'player' in col.lower()]
    print(f"Possible ID columns found: {possible_id_columns}")

    if possible_id_columns:
        # Use the first possible ID column
        id_column = possible_id_columns[0]
        initial_count = len(df_filtered)
        df_filtered = df_filtered.drop_duplicates(subset=[id_column], keep='first')
        duplicates_removed = initial_count - len(df_filtered)
        print(f"Used '{id_column}' for deduplication. Removed {duplicates_removed} duplicate rows")

print(f"Final dataset shape: {df_filtered.shape}")

# Save the initial cleaned dataset
initial_output = 'Romania_CB_Players_Cleaned.csv'
df_filtered.to_csv(initial_output, index=False)
print(f"Initial cleaned dataset saved as: {initial_output}")

# Load additional CSV files for joining
print("\n=== Loading additional CSV files for joining ===")

try:
    # Load players_faces.csv
    df_faces = pd.read_csv('players_faces.csv')
    print(f"Loaded players_faces.csv: {df_faces.shape}")
    print(f"Faces columns: {list(df_faces.columns)}")

    # Load transfermarkt_players.csv
    df_transfermarkt = pd.read_csv('transfermarkt_players.csv')
    print(f"Loaded transfermarkt_players.csv: {df_transfermarkt.shape}")
    print(f"Transfermarkt columns: {list(df_transfermarkt.columns)}")

    # Rename columns in transfermarkt to lowercase as requested
    df_transfermarkt = df_transfermarkt.rename(columns={
        'Age': 'age',
        'Height': 'height',
        'Foot': 'foot'
    })

    # Identify the playerId column in our main dataset
    player_id_col = 'playerId'
    if 'playerId' not in df_filtered.columns:
        # Find the column we used for deduplication
        possible_id_columns = [col for col in df_filtered.columns if 'id' in col.lower() or 'player' in col.lower()]
        if possible_id_columns:
            player_id_col = possible_id_columns[0]
            print(f"Using '{player_id_col}' as player ID column")

    print(f"\n=== Performing joins on '{player_id_col}' ===")

    # First join with players_faces (left join to keep all CB players)
    df_joined = df_filtered.merge(df_faces[['playerId', 'url']],
                                 left_on=player_id_col,
                                 right_on='playerId',
                                 how='left')

    # Remove duplicate playerId column if it was created
    if player_id_col != 'playerId' and 'playerId' in df_joined.columns:
        df_joined = df_joined.drop('playerId', axis=1)

    print(f"After joining with faces: {df_joined.shape}")
    faces_matched = df_joined['url'].notna().sum()
    print(f"Players with face URLs: {faces_matched}/{len(df_joined)}")

    # Second join with transfermarkt_players
    df_final = df_joined.merge(df_transfermarkt[['playerId', 'age', 'height', 'foot']],
                              left_on=player_id_col,
                              right_on='playerId',
                              how='left')

    # Remove duplicate playerId column if it was created
    if player_id_col != 'playerId' and 'playerId' in df_final.columns:
        df_final = df_final.drop('playerId', axis=1)

    print(f"After joining with transfermarkt: {df_final.shape}")
    age_matched = df_final['age'].notna().sum()
    height_matched = df_final['height'].notna().sum()
    foot_matched = df_final['foot'].notna().sum()
    print(f"Players with age data: {age_matched}/{len(df_final)}")
    print(f"Players with height data: {height_matched}/{len(df_final)}")
    print(f"Players with foot data: {foot_matched}/{len(df_final)}")

    # Reorder columns as requested:
    # Insert 'url' between 4th and 5th columns (index 3 and 4)
    # Insert 'age', 'height', 'foot' after the 5th column (which becomes index 5 after url insertion)

    cols = list(df_final.columns)

    # Remove the new columns from their current positions
    new_cols = []
    for col in cols:
        if col not in ['url', 'age', 'height', 'foot']:
            new_cols.append(col)

    # Insert url between 4th and 5th columns (index 4)
    if len(new_cols) >= 4:
        new_cols.insert(4, 'url')
        # Insert age, height, foot after position 5 (which is now index 5)
        new_cols.insert(6, 'age')
        new_cols.insert(7, 'height')
        new_cols.insert(8, 'foot')
    else:
        # If there are fewer than 4 original columns, append at the end
        new_cols.extend(['url', 'age', 'height', 'foot'])

    # Reorder the dataframe
    df_final = df_final[new_cols]

    # Remove duplicates from the final dataset (keeping first occurrence)
    print(f"\n=== Removing duplicates from final dataset ===")
    initial_final_count = len(df_final)
    df_final = df_final.drop_duplicates(subset=[player_id_col], keep='first')
    final_duplicates_removed = initial_final_count - len(df_final)
    print(f"Removed {final_duplicates_removed} duplicate rows from final dataset")

    # Remove the last 2 columns
    print(f"\n=== Removing last 2 columns ===")
    if len(df_final.columns) >= 2:
        cols_before_removal = list(df_final.columns)
        last_two_cols = cols_before_removal[-2:]
        df_final = df_final.iloc[:, :-2]  # Remove last 2 columns
        print(f"Removed columns: {last_two_cols}")
        print(f"Columns remaining: {len(df_final.columns)}")
    else:
        print("Warning: Dataset has fewer than 2 columns, cannot remove last 2 columns")

    # Save the final joined dataset
    final_output = 'Romania_CB_Players_Final.csv'
    df_final.to_csv(final_output, index=False)
    print(f"\nFinal joined dataset saved as: {final_output}")

    # Display final dataset info
    print(f"\n=== Final dataset info ===")
    print(f"Final shape: {df_final.shape}")
    print(f"Total CB players: {len(df_final)}")

    # Show first few rows
    print("\nFirst 3 rows of final dataset:")
    print(df_final.head(3))

    # Show column names for reference
    print(f"\nFinal column order:")
    for i, col in enumerate(df_final.columns):
        print(f"{i+1:2d}. {col}")

    print(f"\nData completeness:")
    print(f"- Players with face URLs: {df_final['url'].notna().sum()}/{len(df_final)} ({df_final['url'].notna().sum()/len(df_final)*100:.1f}%)")
    print(f"- Players with age data: {df_final['age'].notna().sum()}/{len(df_final)} ({df_final['age'].notna().sum()/len(df_final)*100:.1f}%)")
    print(f"- Players with height data: {df_final['height'].notna().sum()}/{len(df_final)} ({df_final['height'].notna().sum()/len(df_final)*100:.1f}%)")
    print(f"- Players with foot data: {df_final['foot'].notna().sum()}/{len(df_final)} ({df_final['foot'].notna().sum()/len(df_final)*100:.1f}%)")

except FileNotFoundError as e:
    print(f"Error: Could not find one of the CSV files: {e}")
    print("Please make sure 'players_faces.csv' and 'transfermarkt_players.csv' are in the same directory")

    # Still save the initial cleaned dataset
    print(f"Initial cleaned dataset is still available as: {initial_output}")

except Exception as e:
    print(f"Error during joining process: {e}")
    print(f"Initial cleaned dataset is still available as: {initial_output}")

Original dataset shape: (1066, 271)
Columns: ['playerId', 'shortName', 'firstName', 'lastName', 'birthDate', 'birthArea.name', 'passportArea.name', 'generic.position.code', 'teams.name', 'competitionId', 'seasonId', 'positions.percent', 'positions.position.name', 'positions.position.code', 'total.matches', 'total.matchesInStart', 'total.matchesSubstituted', 'total.matchesComingOff', 'total.minutesOnField', 'total.minutesTagged', 'total.goals', 'total.assists', 'total.shots', 'total.headShots', 'total.yellowCards', 'total.redCards', 'total.directRedCards', 'total.penalties', 'total.linkupPlays', 'total.duels', 'total.duelsWon', 'total.defensiveDuels', 'total.defensiveDuelsWon', 'total.offensiveDuels', 'total.offensiveDuelsWon', 'total.aerialDuels', 'total.aerialDuelsWon', 'total.fouls', 'total.passes', 'total.successfulPasses', 'total.smartPasses', 'total.successfulSmartPasses', 'total.passesToFinalThird', 'total.successfulPassesToFinalThird', 'total.crosses', 'total.successfulCrosses',